In [2]:
import random
import threading
import time


In [3]:
class Order:
    def __init__(self, order_type, ticker, quantity, price):
        self.order_type = order_type  # "Buy" or "Sell"
        self.ticker = ticker          # e.g. "T123"
        self.quantity = quantity
        self.price = price

    def __str__(self):
        return f"Order({self.order_type}, {self.ticker}, Q:{self.quantity}, P:{self.price})"

class OrderBook:
    def __init__(self):
        # Lists of orders for the given ticker.
        self.buy_orders = []   # Buy orders list
        self.sell_orders = []  # Sell orders list



In [4]:
# Preallocate an array of 1024 order books.
# Each index corresponds to a ticker “T<index>” where index in [0,1023]
order_books = [OrderBook() for _ in range(1024)]


In [5]:

def getTickerIndex(ticker):
    """
    Convert ticker symbol (assumed to be of the form "T<number>") 
    to an index for order_books.
    """
    # Example: ticker "T500" returns integer 500.
    return int(ticker[1:])

# ----------------------------
# Order handling functions
# ----------------------------

def addOrder(order_type, ticker, quantity, price):
    """
    Add an order to the order book.
    
    Parameters:
      order_type (str): "Buy" or "Sell"
      ticker (str): Ticker symbol (e.g., "T123")
      quantity (int): Number of shares
      price (float): Price per share
    """
    index = getTickerIndex(ticker)
    order = Order(order_type, ticker, quantity, price)
    # Append the order to the proper list (simulate lock-free insertion)
    if order_type == "Buy":
        order_books[index].buy_orders.append(order)
    elif order_type == "Sell":
        order_books[index].sell_orders.append(order)
    print("Added order:", order)

def matchOrder(ticker):
    """
    Match Buy and Sell orders for a given ticker.
    
    Matching criteria:
      - If the best (lowest) Sell order price is less than or equal to a Buy order's price,
        the orders are matched.
      - Partial fills are supported.
      
    Time complexity: O(n) where n is the number of orders in the ticker’s order book.
    """
    index = getTickerIndex(ticker)
    book = order_books[index]
    
    # Continue matching as long as a valid match exists.
    while True:
        # Find the lowest-priced Sell order (linear scan: O(n))
        lowest_sell = None
        lowest_sell_index = -1
        for i, sell in enumerate(book.sell_orders):
            if sell is not None:
                if lowest_sell is None or sell.price < lowest_sell.price:
                    lowest_sell = sell
                    lowest_sell_index = i
        
        # No sell orders available → nothing to match
        if lowest_sell is None:
            break
        
        # Look for a Buy order that can match the lowest Sell order.
        matched = False
        for j, buy in enumerate(book.buy_orders):
            if buy is not None and buy.price >= lowest_sell.price:
                # Determine traded quantity (supporting partial fills)
                traded_quantity = min(buy.quantity, lowest_sell.quantity)
                print("Matching:", buy, "with", lowest_sell, f"for Q:{traded_quantity}")
                
                # Update quantities after trade
                buy.quantity -= traded_quantity
                lowest_sell.quantity -= traded_quantity
                
                # Mark orders for removal if fully filled.
                if buy.quantity == 0:
                    book.buy_orders[j] = None
                if lowest_sell.quantity == 0:
                    book.sell_orders[lowest_sell_index] = None
                
                matched = True
                break
        

        if not matched:
            break

        book.buy_orders = [o for o in book.buy_orders if o is not None]
        book.sell_orders = [o for o in book.sell_orders if o is not None]



In [6]:
running = True

def simulate_add_orders():
    """
    Simulate active stock transactions by randomly adding orders.
    """
    while running:
        order_type = random.choice(["Buy", "Sell"])
        # Generate a ticker from T0 to T1023.
        ticker = "T" + str(random.randint(0, 1023))
        quantity = random.randint(1, 100)
        price = round(random.uniform(10, 1000), 2)
        addOrder(order_type, ticker, quantity, price)
        # Sleep briefly to simulate real-time order arrivals.
        time.sleep(0.01)

def simulate_match_orders():
    """
    Continuously try to match orders for random tickers.
    """
    while running:
        ticker = "T" + str(random.randint(0, 1023))
        matchOrder(ticker)
        # Sleep briefly between matching attempts.
        time.sleep(0.01)



In [7]:
# ----------------------------
# Start the simulation with multiple threads.
# ----------------------------

threads = []
t1 = threading.Thread(target=simulate_add_orders)
t2 = threading.Thread(target=simulate_match_orders)
threads.append(t1)
threads.append(t2)

t1.start()
t2.start()

# Run simulation for a fixed period (e.g., 5 seconds)
time.sleep(5)
running = False

# Wait for threads to finish
for t in threads:
    t.join()

print("Simulation complete.")


Added order: Order(Sell, T812, Q:55, P:946.72)
Added order: Order(Sell, T845, Q:1, P:793.49)
Added order: Order(Sell, T743, Q:87, P:84.34)
Added order: Order(Buy, T841, Q:27, P:967.77)
Added order: Order(Sell, T542, Q:82, P:587.28)
Added order: Order(Sell, T365, Q:23, P:276.29)
Added order: Order(Sell, T343, Q:24, P:482.49)
Added order: Order(Sell, T42, Q:19, P:941.66)
Added order: Order(Sell, T714, Q:83, P:854.07)
Added order: Order(Sell, T1022, Q:14, P:946.31)
Added order: Order(Buy, T466, Q:51, P:84.05)
Added order: Order(Buy, T755, Q:7, P:54.1)
Added order: Order(Sell, T832, Q:24, P:828.05)
Added order: Order(Buy, T1007, Q:48, P:481.39)
Added order: Order(Sell, T697, Q:96, P:450.87)
Added order: Order(Sell, T298, Q:52, P:346.94)
Added order: Order(Buy, T340, Q:67, P:121.47)
Added order: Order(Sell, T99, Q:25, P:770.24)
Added order: Order(Sell, T912, Q:37, P:463.91)
Added order: Order(Buy, T649, Q:46, P:758.65)
Added order: Order(Sell, T235, Q:18, P:303.69)
Added order: Order(Sell, 